# Decision trees - The CART algorithm

In this project, we will be programming our own decision tree CART algorithm with Gini criterion. 


To evaluate the algorithm we will be working with the [Early stage diabetes risk prediction dataset](https://archive.ics.uci.edu/ml/datasets/Early+stage+diabetes+risk+prediction+dataset.) containing predictor variables about patients with diabetes and controls. 


**0. Moving the lab to your folder**

Copy this notebook to your personal directory

In [121]:
# Library import.
import urllib.request
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np

**1. Data Loading**

In [122]:
sData='http://archive.ics.uci.edu/ml/machine-learning-databases/00529/diabetes_data_upload.csv'
urllib.request.urlretrieve(sData,'./datafile.csv') # The data is stored in your drive folder under the name 'datafile.csv'
df=pd.read_csv('./datafile.csv')

In [123]:
df = df.rename(columns={"class": "Class"}) #Renaming column name 'class' to 'Class'
df.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,Class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In [124]:
series=df.iloc[3,:-1] # The last column is removed (it's the class)
print(series)

Age                     45
Gender                Male
Polyuria                No
Polydipsia              No
sudden weight loss     Yes
weakness               Yes
Polyphagia             Yes
Genital thrush         Yes
visual blurring         No
Itching                Yes
Irritability            No
delayed healing        Yes
partial paresis         No
muscle stiffness        No
Alopecia                No
Obesity                 No
Name: 3, dtype: object


**2. Dataset descriptors**
**Data and notation used in the algorithm:**
X is the training subset containing N observations
Each observation, x_n is a vector containing the predictor variables (size M) for each observation.
y_n is the label of x_n
J: total number of classes
N: total number of observations (number of training vectors)
M: number of predictor variables (feature vector length)

**TASK 1:** In the next cell, we calculate N, M and J.

In [125]:
#First steps: Determine the number of features, classes and observations 
N=len(df) # No. of Observations
M=len(df.columns[:-1]) # No. of predictor variables
J=len(df.Class.unique()) # No. of classes

print('Number of observations: '+str(N))
print('Number of predictor variables: '+str(M))
print('Number of classes: '+str(J))

Number of observations: 520
Number of predictor variables: 16
Number of classes: 2


**TASK 2:** Splitting the data into, approx, 80% training and 20% testing (choosing the first 80% observations as training and the rest as testing)

In [126]:
# Split the data into, approx, 80% training and 20% testing
dfTrain=df.head(n=int((len(df)*0.8))) # Training data set
dfTest = df[max(dfTrain.index)+1:] # Testing data set


# 3. The CART Algorithm
*1.	Find each predictor’s best split:*

* a.	Determine if the predictor is categorical or numerical


* b.	For each predictor, obtain unique values. 

* c.	For each predictor: using the unique values, go through each value to examine each predictor’s possible split point


* - i.	The best split point (threshold) is the one that maximizes the splitting criterion1 (impurity gain) the most.

* - ii.	In categorical predictors, we will have to consider a possible node for each category.


*2.	Find the predictor that provides the best split:*

* a.	Select the predictor that maximizes the splitting criterion (the one that produces the highest impurity gain.


*3.	Split the data X into two new nodes using the split point calculated in point 1 for the predictor selected in point 2.*


*4.	Repeat point 1 for each branch if stopping rules are not satisfied:*

* a.	Stopping rules: 
* * i.	Maximum tree depth (maximum number of nodes in a row)
* * ii.	All the remaining training observations in a resulting branch belong to a single class (maximum purity)
* * iii.	All observations in a node have the same predictor’s values: it is not possible to split more. 
* * iv.	The node size is less than the minimum node size (number of observations in a node)

In the following cells, creating the functions required for each point.


**TASK 3:** 1.a.  Determining if the predictor is categorical or numerical

In [127]:
# Finding if the predictor is categorical or numerical
def is_numerical(df_inp):
  is_numeric = [];
  for i in range(len(df_inp.columns)):
    is_numeric.append(is_numeric_dtype(df_inp.iloc[:,i]))
  return is_numeric

In [128]:
# Example
is_numerical(dfTrain.iloc[:,:-1])

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

**TASK 4:** 1.b. b.	For each predictor, obtaining unique values (categories)

In [129]:
#1.b Function to obtain categories per predictor 
def unique_values(df_inp,column):
  categories = df_inp[column].unique()
  return categories

In [130]:
#Example
categories=unique_values(df,'Gender')
print(categories)
print(categories[0])

['Male' 'Female']
Male


**TASK 5:** 1.c.	For each predictor: using the unique values, going through each value to examine each predictor’s possible split point

1. c. i. Searching for the best split point (threshold), the one that maximizes the splitting criterion1 (impurity gain) the most.

In [131]:
#1.c.i Generate the gini function that calculates impurity 
# Calculating impurity
def impurity_gini(df_inp):
  uni = df_inp[df_inp.columns[-1]].unique()
  impurity = 0
  if len(df_inp)==0:
    postitive = negative = 0
  else:
    positive = len(df_inp[df_inp['Class']==uni[0]])/len(df_inp)
    negative = len(df_inp[df_inp['Class']!=uni[0]])/len(df_inp)
    impurity = 1 - (positive)**2-(negative)**2
  return impurity

# Calculating Gain
def impurity_gain(df_inp, dfl, dfr):
  impurity_total = impurity_gini(df_inp)
  impurity_L = impurity_gini(dfl)
  impurity_R = impurity_gini(dfr)
  wL =len(dfl)/len(df_inp)
  wR = len(dfr)/len(df_inp)
  gain = impurity_total - (wL*impurity_L) - (wR*impurity_R)
  return gain

In [132]:
impurity_gini(df)

0.47337278106508873

Expected output: 0.47337278106508873

In [133]:
impurity_gain(df,df, df)

-0.47337278106508873

**TASK 6:** Searching for the split point (or threshold) that provides the highest impurity gain in numerical predictor variables (for instance, 'Age' is a numerical variable).

In [204]:
#1.c.i Numerical split analysis 
# Spliting Numerical Predictor
from cmath import inf


def numerical_max_search(df_inp, column):
  ig_max = -inf
  df_inp = df_inp.sort_values(by=[column])
  for i in df_inp[column].unique():
    dfl = df_inp[df_inp[column]<=i]
    dfr = df_inp[df_inp[column]>i]
    gain = impurity_gain(df_inp,dfl, dfr)
    if gain > ig_max:
      ig_max = gain
      category_max = i
  return ig_max, category_max

In [205]:
numerical_max_search(df, 'Age')

(0.015152664201645849, 34)

**TASK 7:** 1.c.	For each predictor: using the unique values, going through each value to examine each predictor’s possible split point

1. c. ii.	In categorical predictors, we will have to consider a possible node for each category. This means that we have to perform a search of best split for each category. In the dataset that we are using, each of the categorical variables have only two possible categories (Yes or No - Male or Female). 

In [206]:
# 1.c.ii Categorical split analysis (10 POINTS)
# Spliting Categorical Predictor
from cmath import inf


def categorical_max_search(df_inp, column):
  max = -inf
  ig_max = 0
  category_max = df_inp[column].unique()[0]

  df_inp = df_inp.sort_values(by=[column])
  df_inp[column].unique()
  if len(df_inp[column].unique())>=2:
    for x in df_inp[column].unique():
      dfl = df_inp[df_inp[column]==x]
      dfr = df_inp[df_inp[column]!=x]
      if max < impurity_gain(df_inp,dfl, dfr):
         max = impurity_gain(df_inp,dfl, dfr)
         category_max = x
         ig_max = max
  else:
    category_max = df_inp[column].unique()[0]
    dfl = df_inp[df_inp[column]==category_max]
    dfr = df_inp[df_inp[column]!=category_max]
    max = impurity_gain(df_inp,dfl, dfr)
    ig_max = max
  return ig_max, category_max

In [207]:
categorical_max_search(df, 'Polydipsia')

(0.199221516230269, 'No')

**TASK 8:** 2.	Using the previous functions, finding the predictor that provides the best split: 
a.	Selecting the predictor that maximizes the splitting criterion (the one that produces the highest impurity gain).


In [208]:
# 2.a Find predictor that provides best split 
# Finding predictor(Categorical or Numerical) that provides best split
def find_max_predictor(df_inp):
  Max_ig = 0
  category = is_numerical(df_inp)
  for i in range(len(category)-1):
    if category[i]==False:
      a,b = categorical_max_search(df_inp, df_inp.columns[i])
      if a > Max_ig:
        Max_ig = a
        Max_predictor = df_inp.columns[i]
        Max_category = b
    else:
      a,b = numerical_max_search(df_inp, df_inp.columns[i])
      if a > Max_ig:
        Max_ig = a
        Max_predictor = df_inp.columns[i]
        Max_category = b

  return Max_ig, Max_predictor, Max_category

In [209]:
# Example
Max_ig, Max_predictor, Max_category=find_max_predictor(df)
print('Maximum Impurity Gain: ' +str(Max_ig) +' obtained with the predictor '+ Max_predictor +' and category ' +str(Max_category))

Maximum Impurity Gain: 0.20991841189440502 obtained with the predictor Polyuria and category No


**TASK 9:** 3.	Spliting any input data X (a dataframe) into two new nodes using the 
split point and predictor selected obtained with find_max_predictor.

In [210]:
# 3. Split data
# Spliting the data into nodes
def split_data(df_inp):
  Max_ig, Max_predictor, Max_category=find_max_predictor(df_inp)
  if is_numeric_dtype('Max_predictor')==True:
    dfL = df_inp[df_inp[Max_predictor]<=Max_category]
    dfR = df_inp[df_inp[Max_predictor]>Max_category]
  else:
    dfL = df_inp[df_inp[Max_predictor]==Max_category]
    dfR = df_inp[df_inp[Max_predictor]!=Max_category]
    dfL.drop([Max_predictor],axis=1)
    if len(dfR[Max_predictor].unique()==1):
      dfR.drop(columns=[Max_predictor])
  return dfL,dfR, Max_predictor, Max_category

In [211]:
# This defines the first node of the tree for the df dataset
dfL,dfR,predictor,treshold=split_data(df)
# This calculates the Impurity Gain when going from the initial dataset (df) to 
# the left and right subsets
impurity_gain(df,dfL,dfR)

0.20991841189440502

**Additional code:**
The following function and classes serve to store information of the nodes and leaves.

In [212]:
def classify_leaf(df_inp):
  """ 
  Counts the number of observations per class (class probability) in the input df 
  This function serves to create the classification info stored in the leaf.
  """

  classes=df_inp.iloc[:,-1] # Labels in df
  classes_uni=classes.unique() # this variable contains the classes that exist in df
  result={Tclass:(sum(classes==Tclass)/len(df_inp)) for Tclass in classes_uni} #proportion of observations per class
  return result

class tree_leaf:
    """
    This class will contain the results of the leaf (ex: Positive: 0.8, Negative:0.2)
    """

    def __init__(self, df_inp):
        self.predictions = classify_leaf(df_inp)

class tree_node:
    """
    This class will contain the predictor and treshold of the node and a the two 
    nested nodes (left and right)
    """

    def __init__(self,
                 predictor,
                 threshold,
                 left_branch,
                 right_branch):
        self.predictor = predictor
        self.threshold = threshold
        self.left_branch = left_branch
        self.right_branch = right_branch

**TASK 10:** Stopping rules. These rules will serve to decide when to stop looking for a node and create a leaf. Conventional algorithms might have more stopping rules, but we will restrict our rules to only four conditions. We will code the functions that check these rules.

4.a.	Stopping rules:

i.	Maximum tree depth (maximum number of nodes in a row) (this will be checked in the main function)

ii.	All the remaining training observations in a resulting branch belong to a single class (maximum purity)

iii.	All observations in a node have the same predictor’s values: it is not possible to split more. 

iv.	The node size is less than the minimum node size (number of observations in a node)


In [213]:
# check stopping rules 

#4.a.ii
# Checking if all the remaining training obervations in a resulting branch belong to a single class
def check_impurity(df_inp):
  #returns TRUE if immpurity of the data in df is equal to 0.
  if impurity_gini(df_inp)==0:
    return True
  else:
    return False

#4.a.iii
#Checking if it is not possible to split anymore
def check_predictor(df_inp):
  return len(df_inp.Class.unique()) ==1 
  
  
#4.a.iv
#Checking if node size is less than minimum node size
def check_node_length(df_inp, min_partition_length):
  #returns TRUE if the number of observations in df is less than min_partition_length
    if len(df_inp)<min_partition_length:
      return True
    else:
      return False

**End: Train the tree**

The following functions should train and evaluate a decision tree. 

**TASK 11:** Coding the training of the right (false) branch

In [214]:
# train the tree  
def train_tree_mlma(df_inp, tree_depth=0, min_partition_length=2, max_tree_depth=200):

  dfL,dfR,predictor,threshold=split_data(df_inp)
  tree_depth+=1 #the tree depth in this path is extended
  print('Tree depth: ' +str(tree_depth))
  #tree_depth==max_tree_depth checks condition 4.a.i
  #Left branch (true)
  if check_impurity(dfL) or check_predictor(dfL) or check_node_length(dfL,min_partition_length) or tree_depth==max_tree_depth:
    
    left_node=tree_leaf(dfL)
  else:
    left_node=train_tree_mlma(dfL, tree_depth)
  
  #Right branch (false)
  if check_impurity(dfR) or check_predictor(dfR) or check_node_length(dfR,min_partition_length) or tree_depth==max_tree_depth:
    
    right_node=tree_leaf(dfR)
  else:
    right_node=train_tree_mlma(dfR, tree_depth)



  return tree_node(predictor, threshold, left_node, right_node)

In [215]:
my_tree=train_tree_mlma(dfTrain)

Tree depth: 1
Tree depth: 2
Tree depth: 3
Tree depth: 4
Tree depth: 5
Tree depth: 6
Tree depth: 7
Tree depth: 6
Tree depth: 7
Tree depth: 8
Tree depth: 9
Tree depth: 5
Tree depth: 6
Tree depth: 4
Tree depth: 5
Tree depth: 6
Tree depth: 5
Tree depth: 6
Tree depth: 7
Tree depth: 3
Tree depth: 4
Tree depth: 5
Tree depth: 6
Tree depth: 7
Tree depth: 8
Tree depth: 4
Tree depth: 2
Tree depth: 3
Tree depth: 4
Tree depth: 5
Tree depth: 6
Tree depth: 7
Tree depth: 8
Tree depth: 9
Tree depth: 10
Tree depth: 11
Tree depth: 12
Tree depth: 13
Tree depth: 14
Tree depth: 15
Tree depth: 16
Tree depth: 17
Tree depth: 18
Tree depth: 19
Tree depth: 20
Tree depth: 21
Tree depth: 9


In [216]:
def print_tree_mlma(my_tree, jump=''):
  jump=jump+'-'
  if isinstance(my_tree, tree_leaf):
    print(jump+'Result: ')
    print(my_tree.predictions)
  else:
    if is_numeric_dtype(my_tree.threshold):
      print(jump + '> If ' + str(my_tree.predictor) + ' <= ' + str(my_tree.threshold))
      print_tree_mlma(my_tree.left_branch,jump)
      print(jump+'> Else: ')
      print_tree_mlma(my_tree.right_branch,jump)
    
    else:
      print(jump + '> If ' + str(my_tree.predictor) + ' is ' + str(my_tree.threshold))
      print_tree_mlma(my_tree.left_branch,jump)
      print(jump+'> Else: ')
      print_tree_mlma(my_tree.right_branch,jump)

In [217]:
print_tree_mlma(my_tree)

-> If Polyuria is No
--> If Gender is Male
---> If Polydipsia is No
----> If Irritability is No
-----> If partial paresis is Yes
------> If muscle stiffness is No
-------> If Itching is No
--------Result: 
{'Positive': 1.0}
-------> Else: 
--------Result: 
{'Negative': 1.0}
------> Else: 
-------Result: 
{'Negative': 1.0}
-----> Else: 
------> If delayed healing is No
-------Result: 
{'Negative': 1.0}
------> Else: 
-------> If Age <= 37
--------Result: 
{'Positive': 1.0}
-------> Else: 
--------> If Alopecia is No
---------> If Age <= 45
----------Result: 
{'Positive': 1.0}
---------> Else: 
----------Result: 
{'Negative': 1.0}
--------> Else: 
---------Result: 
{'Negative': 1.0}
----> Else: 
-----> If Genital thrush is No
------> If Age <= 42
-------Result: 
{'Positive': 1.0}
------> Else: 
-------Result: 
{'Negative': 1.0}
-----> Else: 
------Result: 
{'Positive': 1.0}
---> Else: 
----> If muscle stiffness is No
-----> If partial paresis is No
------Result: 
{'Positive': 1.0}
----->

In [218]:
def classify_tree_mlma(my_tree, my_series):
  """
  Classifies the input series and returns the predictions
  """
  if isinstance(my_tree, tree_leaf):
    return my_tree.predictions
  else:
    if is_numeric_dtype(my_tree.threshold): # Numeric
      if my_series[my_tree.predictor] <= my_tree.threshold:
        return classify_tree_mlma(my_tree.left_branch,my_series)
      else:
        return classify_tree_mlma(my_tree.right_branch,my_series)
    else: #Categorical
      if my_series[my_tree.predictor] == my_tree.threshold:
        return classify_tree_mlma(my_tree.left_branch,my_series)
      else:
        return classify_tree_mlma(my_tree.right_branch,my_series)


      
 

In [86]:
# Predictions
my_series=dfTest.iloc[33,:]
prediction=classify_tree_mlma(my_tree, my_series )
print('True class: ' + my_series[-1] + ' - ' + my_series.keys()[-1])
print('Predicted class: ')
print(prediction)

True class: Positive - Class
Predicted class: 
{'Positive': 1.0}



Coding a function that calculates the accuracy of the trained tree when classifying the whole dfTest subset.

In [87]:
# Forming list of predicted and actual values
actual = pd.Series(dfTest['Class']).values
temp = pd.Series()
for x in range(len(dfTest)):
  my_series=dfTest.iloc[x,:]
  prediction=classify_tree_mlma(my_tree, my_series )
  a = pd.Series(prediction)
  temp = temp.append(a)
predicted = temp.index.values

C:\Users\cairu\AppData\Local\Temp/ipykernel_10020/1116971243.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  temp = pd.Series()


In [219]:
# Calculating accuracy
def accuracy(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0
 
accuracy = accuracy(actual, predicted)
print(accuracy)

96.15384615384616
